#  การทดลองเสริม: ค่าใช้จ่ายแบบมีการปรับ (Regularized Cost) และ การไล่ระดับ

## เป้าหมาย
ในแล็บนี้ คุณจะ:
- ขยายฟังก์ชันต้นทุนเชิงเส้นและโลจิสติกด้วยเทอมการปรับสมดุล (regularization term)
- เรียกใช้ตัวอย่างก่อนหน้าของการโอเวอร์ฟิตติ้งอีกครั้งโดยเพิ่มเทอมการปรับสมดุล

In [ ]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from plt_overfit import overfit_example, output
from lab_utils_common import sigmoid
np.set_printoptions(precision=8)

# การเพิ่ม Regularization (การทำให้เป็นปกติ)
<img align="Left" src="./images/C1_W3_LinearGradientRegularized.png"  style=" width:400px; padding: 10px; " >
<img align="Center" src="./images/C1_W3_LogisticGradientRegularized.png"  style=" width:400px; padding: 10px; " >

สไลด์ข้างบนแสดงฟังก์ชันค่าใช้จ่ายและการไล่ระดับสำหรับทั้งการถดถอยเชิงเส้นและการถดถอยโลจิสติก โปรดสังเกต:
- ฟังก์ชันค่าใช้จ่าย (Cost function):
    - ฟังก์ชันค่าใช้จ่ายของการถดถอยเชิงเส้นและโลจิสติกมีความแตกต่างกันอย่างมาก แต่การเพิ่มการปรับปรุง (regularization) ให้กับสมการนั้นเหมือนกัน

- ฟังก์ชันการไล่ระดับ (Gradient function):
    - ฟังก์ชันการไล่ระดับของการถดถอยเชิงเส้นและโลจิสติกมีความคล้ายคลึงกันมาก $f_{wb}$ เท่านั้น

## ฟังก์ชันต้นทุน (Cost functions) ที่มีการปรับค่า (regularization)
### ฟังก์ชันต้นทุนสำหรับการถดถอยเชิงเส้นแบบปรับค่า (regularized linear regression)


สมการสำหรับฟังก์ชันต้นทุนการถดถอยเชิงเส้นแบบปรับค่ามีดังนี้:
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2  + \frac{\lambda}{2m}  \sum_{j=0}^{n-1} w_j^2 \tag{1}$$ 
โดยที่:
$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b  \tag{2} $$ 


เมื่อเทียบกับฟังก์ชันต้นทุนที่ไม่มีการปรับค่า (ซึ่งคุณได้ใช้งานใน lab ก่อนหน้านี้) จะมีรูปแบบดังนี้:

$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 $$ 

ความแตกต่างอยู่ที่เทอมการปรับค่า (regularization term) ซึ่งมีสีน้ำเงิน:,  <span style="color:blue">
    $\frac{\lambda}{2m}  \sum_{j=0}^{n-1} w_j^2$ </span> 
    
การรวมเทอมนี้เข้าไปจะช่วยให้การไล่ระดับ (gradient descent) ลดขนาดของพารามิเตอร์ได้มากขึ้น โปรดสังเกตว่าในตัวอย่างนี้ พารามิเตอร์ $b$ ไม่ถูกปรับค่า นี่เป็นวิธีปฏิบัติมาตรฐาน


ด้านล่างนี้เป็นการใช้งานสมการ (1) และ (2) โปรดสังเกตว่าใช้ **รูปแบบมาตรฐานสำหรับหลักสูตรนี้** ซึ่งเป็น  `for loop` ที่วนซ้ำสำหรับตัวอย่างทั้งหมด m ตัว


In [ ]:
def compute_cost_linear_reg(X, y, w, b, lambda_ = 1):
    """
    Computes the cost over all examples
    Args:
      X (ndarray (m,n): Data, m examples with n features
      y (ndarray (m,)): target values
      w (ndarray (n,)): model parameters  
      b (scalar)      : model parameter
      lambda_ (scalar): Controls amount of regularization
    Returns:
      total_cost (scalar):  cost 
    """

    m  = X.shape[0]
    n  = len(w)
    cost = 0.
    for i in range(m):
        f_wb_i = np.dot(X[i], w) + b                                   #(n,)(n,)=scalar, see np.dot
        cost = cost + (f_wb_i - y[i])**2                               #scalar             
    cost = cost / (2 * m)                                              #scalar  
 
    reg_cost = 0
    for j in range(n):
        reg_cost += (w[j]**2)                                          #scalar
    reg_cost = (lambda_/(2*m)) * reg_cost                              #scalar
    
    total_cost = cost + reg_cost                                       #scalar
    return total_cost                                                  #scalar

รันเซลล์ด้านล่างเพื่อดูการทำงานจริง

In [ ]:
np.random.seed(1)
X_tmp = np.random.rand(5,6)
y_tmp = np.array([0,1,0,1,0])
w_tmp = np.random.rand(X_tmp.shape[1]).reshape(-1,)-0.5
b_tmp = 0.5
lambda_tmp = 0.7
cost_tmp = compute_cost_linear_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)

print("Regularized cost:", cost_tmp)

**ผลลัพธ์ที่คาดหวัง**:
<table>
  <tr>
    <td> <b>Regularized cost: </b> 0.07917239320214275 </td>
  </tr>
</table>

### ฟังก์ชันต้นทุนสำหรับการถดถอยโลจิสติกแบบมีการปรับมาตรฐาน (regularized logistic regression)

สำหรับการถดถอยโลจิสติกแบบมีการปรับมาตรฐาน (regularized logistic regression) ฟังก์ชันต้นทุน (cost function) มีรูปแบบดังนี้:

$$J(\mathbf{w},b) = \frac{1}{m}  \sum_{i=0}^{m-1} \left[ -y^{(i)} \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) \right] + \frac{\lambda}{2m}  \sum_{j=0}^{n-1} w_j^2 \tag{3}$$
โดยที่:

$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = sigmoid(\mathbf{w} \cdot \mathbf{x}^{(i)} + b)  \tag{4} $$ 

เมื่อเปรียบเทียบกับฟังก์ชันต้นทุนที่ไม่มีการปรับมาตรฐาน (ซึ่งคุณได้ใช้งานใน lab ก่อนหน้านี้)

$$ J(\mathbf{w},b) = \frac{1}{m}\sum_{i=0}^{m-1} \left[ (-y^{(i)} \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right)\right] $$

ความแตกต่างอยู่ที่เทอมการปรับมาตรฐาน (regularization term) ซึ่งคือ:
   <span style="color:blue">
    $\frac{\lambda}{2m}  \sum_{j=0}^{n-1} w_j^2$ </span> 

การรวมเทอมนี้เข้าไปจะช่วยกระตุ้นให้การไล่ระดับ (gradient descent) ลดขนาดของพารามิเตอร์ (parameters)

โปรดสังเกตว่าในตัวอย่างนี้ พารามิเตอร์ b ไม่ได้ถูกปรับมาตรฐาน เป็นการปฏิบัติตามมาตรฐานทั่วไป

In [ ]:
def compute_cost_logistic_reg(X, y, w, b, lambda_ = 1):
    """
    Computes the cost over all examples
    Args:
    Args:
      X (ndarray (m,n): Data, m examples with n features
      y (ndarray (m,)): target values
      w (ndarray (n,)): model parameters  
      b (scalar)      : model parameter
      lambda_ (scalar): Controls amount of regularization
    Returns:
      total_cost (scalar):  cost 
    """

    m,n  = X.shape
    cost = 0.
    for i in range(m):
        z_i = np.dot(X[i], w) + b                                      #(n,)(n,)=scalar, see np.dot
        f_wb_i = sigmoid(z_i)                                          #scalar
        cost +=  -y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i)      #scalar
             
    cost = cost/m                                                      #scalar

    reg_cost = 0
    for j in range(n):
        reg_cost += (w[j]**2)                                          #scalar
    reg_cost = (lambda_/(2*m)) * reg_cost                              #scalar
    
    total_cost = cost + reg_cost                                       #scalar
    return total_cost                                                  #scalar

รันเซลล์ด้านล่างเพื่อดูการทำงานจริง

In [ ]:
np.random.seed(1)
X_tmp = np.random.rand(5,6)
y_tmp = np.array([0,1,0,1,0])
w_tmp = np.random.rand(X_tmp.shape[1]).reshape(-1,)-0.5
b_tmp = 0.5
lambda_tmp = 0.7
cost_tmp = compute_cost_logistic_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)

print("Regularized cost:", cost_tmp)

**ผลลัพธ์คาดหวัง**:
<table>
  <tr>
    <td> <b>Regularized cost: </b> 0.6850849138741673 </td>
  </tr>
</table>

## การไล่ระดับ (Gradient Descent) พร้อมการปรับสมดุล (Regularization)
อัลกอริทึมพื้นฐานของการไล่ระดับ (gradient descent) จะไม่เปลี่ยนแปลงเมื่อมีการปรับสมดุล (regularization)

$$\begin{align*}
&\text{repeat until convergence:} \; \lbrace \\
&  \; \; \;w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{for j := 0..n-1} \\ 
&  \; \; \;  \; \;b = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \\
&\rbrace
\end{align*}$$
ในอัลกอริทึมการไล่ระดับ (gradient descent) การอัปเดตพารามิเตอร์  $w_j$ จะทำพร้อมกันสำหรับทุกค่า $j$.ในแต่ละรอบการวนซ้ำ (iteration)

เมื่อมีการเพิ่มเทคนิคการปรับปรุง (regularization) สิ่งที่เปลี่ยนแปลงคือการคำนวณการไล่ระดับ (gradients)

### การคำนวณ Gradient พร้อมการปรับสมดุล (Regularization) สำหรับ Linear และ Logistic Regression
การคำนวณ Gradient สำหรับทั้ง Linear Regression และ Logistic Regression นั้นแทบจะเหมือนกัน โดยแตกต่างกันเพียงแค่การคำนวณ  $f_{\mathbf{w}b}$ เท่านั้น
$$\begin{align*}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)}  +  \frac{\lambda}{m} w_j \tag{2} \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{3} 
\end{align*}$$

* m คือจำนวนตัวอย่างการฝึก (training examples) ในชุดข้อมูล      
* $f_{\mathbf{w},b}(x^{(i)})$  คือการทำนายของโมเดล ขณะที่  $y^{(i)}$ คือเป้าหมาย (target)

      
* สำหรับ a  <span style="color:blue"> **linear** </span> regression model  
    $f_{\mathbf{w},b}(x) = \mathbf{w} \cdot \mathbf{x} + b$  
* สำหรับ a <span style="color:blue"> **logistic** </span> regression model  
    $z = \mathbf{w} \cdot \mathbf{x} + b$  
    $f_{\mathbf{w},b}(x) = g(z)$  
    โดยที่ $g(z)$ คือ sigmoid function:  
    $g(z) = \frac{1}{1+e^{-z}}$   
    
เทอมที่ใช้ในการปรับตัวคือ  <span style="color:blue">$\frac{\lambda}{m} w_j $</span>

### ฟังก์ชันการไล่ระดับสำหรับการถดถอยเชิงเส้นแบบมีการปรับค่า (regularized linear regression)

In [ ]:
def compute_gradient_linear_reg(X, y, w, b, lambda_): 
    """
    Computes the gradient for linear regression 
    Args:
      X (ndarray (m,n): Data, m examples with n features
      y (ndarray (m,)): target values
      w (ndarray (n,)): model parameters  
      b (scalar)      : model parameter
      lambda_ (scalar): Controls amount of regularization
      
    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b. 
    """
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]                 
        for j in range(n):                         
            dj_dw[j] = dj_dw[j] + err * X[i, j]               
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m   
    
    for j in range(n):
        dj_dw[j] = dj_dw[j] + (lambda_/m) * w[j]

    return dj_db, dj_dw

รันเซลล์ด้านล่างเพื่อดูการทำงานจริง

In [ ]:
np.random.seed(1)
X_tmp = np.random.rand(5,3)
y_tmp = np.array([0,1,0,1,0])
w_tmp = np.random.rand(X_tmp.shape[1])
b_tmp = 0.5
lambda_tmp = 0.7
dj_db_tmp, dj_dw_tmp =  compute_gradient_linear_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)

print(f"dj_db: {dj_db_tmp}", )
print(f"Regularized dj_dw:\n {dj_dw_tmp.tolist()}", )

**ผลลัพธ์คาดหวัง**
```
dj_db: 0.6648774569425726
Regularized dj_dw:
 [0.29653214748822276, 0.4911679625918033, 0.21645877535865857]
 ```

### ฟังก์ชันการไล่ระดับสำหรับการถดถอยโลจิสติกแบบมีการปรับสมดุล (Regularized Logistic Regression)


In [ ]:
def compute_gradient_logistic_reg(X, y, w, b, lambda_): 
    """
    Computes the gradient for linear regression 
 
    Args:
      X (ndarray (m,n): Data, m examples with n features
      y (ndarray (m,)): target values
      w (ndarray (n,)): model parameters  
      b (scalar)      : model parameter
      lambda_ (scalar): Controls amount of regularization
    Returns
      dj_dw (ndarray Shape (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar)            : The gradient of the cost w.r.t. the parameter b. 
    """
    m,n = X.shape
    dj_dw = np.zeros((n,))                            #(n,)
    dj_db = 0.0                                       #scalar

    for i in range(m):
        f_wb_i = sigmoid(np.dot(X[i],w) + b)          #(n,)(n,)=scalar
        err_i  = f_wb_i  - y[i]                       #scalar
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i,j]      #scalar
        dj_db = dj_db + err_i
    dj_dw = dj_dw/m                                   #(n,)
    dj_db = dj_db/m                                   #scalar

    for j in range(n):
        dj_dw[j] = dj_dw[j] + (lambda_/m) * w[j]

    return dj_db, dj_dw  


รันเซลล์ด้านล่างเพื่อดูการทำงานจริง

In [ ]:
np.random.seed(1)
X_tmp = np.random.rand(5,3)
y_tmp = np.array([0,1,0,1,0])
w_tmp = np.random.rand(X_tmp.shape[1])
b_tmp = 0.5
lambda_tmp = 0.7
dj_db_tmp, dj_dw_tmp =  compute_gradient_logistic_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)

print(f"dj_db: {dj_db_tmp}", )
print(f"Regularized dj_dw:\n {dj_dw_tmp.tolist()}", )

**ผลลัพธ์คาดหวัง**
```
dj_db: 0.341798994972791
Regularized dj_dw:
 [0.17380012933994293, 0.32007507881566943, 0.10776313396851499]
 ```

## รันตัวอย่างการโอเวอร์ฟิตติ้งอีกครั้ง

In [ ]:
plt.close("all")
display(output)
ofit = overfit_example(True)

ในกราฟข้างต้น ให้ลองทดลองการปรับค่า regularization (การควบคุมความซับซ้อนของโมเดล) บนตัวอย่างก่อนหน้า

- การจำแนก (Categorical Regression):
    - ตั้งค่า degree เป็น 6 และ lambda เป็น 0 (ไม่มีการปรับค่า regularization) แล้วทำการ fit ข้อมูล
    - จากนั้น ตั้งค่า lambda เป็น 1 (เพิ่มการปรับค่า regularization) แล้วทำการ fit ข้อมูลอีกครั้ง สังเกตความแตกต่าง
- การถดถอย (Regression):
    - ทำตามขั้นตอนเดียวกันกับการจำแนก
    - ทดลองเปลี่ยนค่า degree และ lambda เพื่อสังเกตผลกระทบต่อโมเดล

## ขอแสดงความยินดี!
คุณได้:
- ตัวอย่างของค่าใช้จ่ายและการไล่ระดับ (gradient) ที่มีการเพิ่มการปรับมาตรฐาน (regularization) สำหรับทั้งการถดถอยเชิงเส้นและการถดถอยโลจิสติก
- พัฒนาทักษะในการเข้าใจว่าการปรับมาตรฐานสามารถช่วยลดการโอเวอร์ฟิต (overfitting) ได้อย่างไร






